# Classificaton Fine-tuning 

La siguiente tarea consiste en entrenar un modelo de HuggingFace (HF) para realizar la _task_ de _classification_. El dataset para entrenar dicho modelo está predefinido. Sin embargo, el modelo, el tokenizador y el trainer pueden ser totalmente personalizados. Es decir, que tendréis que realizar un trabajo de investigación, de prueba y error, para poder ir aprendiendo y ganando destreza con HF. 

Recomendaciones: 
- Durante este proceso, tendréis muchas dudas y encontraréis muchos errores. Tratad de resolverlas primero por vuestra cuenta, enteniendo la causa del error. Después con recursos online. Y, finalmente, siempre está el foro, que puede ser utilizado de forma participativa.
- No dejeis la tarea para el último día. Los modelos tardan en entrenar. Los problemas no se resuelven en la primera iteración.

Finalmente, se pide:
- Limpieza rigurosa en la presentación del notebook.
- El notebook se entrega con todas las celdas ejecutadas.
- Los comentarios (opcionales), mejor sobre el código con '#'. 

Ánimo!

## Dataset

A continuación, descargarás un dataset llamado _glue_ que contiene 392702 filas en el dataset de train y 9815 registros en el dataset de validation_match. 

Lo primero que tendrás que hacer es construir un dataset nuevo, llamado **ds_tarea**, que filtre el anterior dataset para quedarse con los registros que tengan el contenido de la columna _context_  con menos (estrictamente) de 20 caracteres.

In [1]:
from datasets import load_dataset

dataset = load_dataset("glue", "mnli")
dataset

c:\Users\pablo\anaconda3\envs\deep-learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [2]:
# Imprimir los nombres de las columnas para verificar cuál usar
print("Columnas en Train:", dataset['train'].column_names)
print("Columnas en Validation Matched:", dataset['validation_matched'].column_names)

Columnas en Train: ['premise', 'hypothesis', 'label', 'idx']
Columnas en Validation Matched: ['premise', 'hypothesis', 'label', 'idx']


In [3]:
# Filtrar el dataset para quedarse con los registros donde la longitud de 'context' es menor de 20 caracteres
ds_tarea_train = dataset['train'].filter(lambda x: len(x['premise']) < 20)
ds_tarea_validation = dataset['validation_matched'].filter(lambda x: len(x['premise']) < 20)

# Crear un nuevo dataset combinando los subsets filtrados
ds_tarea = {
    'train': ds_tarea_train,
    'validation_match': ds_tarea_validation
}

In [4]:
assert len(ds_tarea['train']) == 13635
assert len(ds_tarea['validation_match']) == 413

## EDA

Si tenéis que realizar alguna exploración del datos, utilizad esta sección.

In [5]:
# Celdas de libre uso

## Feature Engineering

Si tenéis que realizar alguna modificación de los datos (no siempre es necesaria, pero algunos modelos preentrenados lo piden), podéis utilizar esta sección. 

Al finalizar la sección, bien si modificais el dataset, bien si no lo modificáis, lo guardaréis en un dataset llamado __ds_tarea_featured__.

In [6]:
# Celdas de libre uso

In [5]:
ds_tarea_featured = ds_tarea # Esta línea tiene sentido en caso de que no se modifique el dataset

In [7]:
assert len(ds_tarea_featured['train']) == 13635
assert len(ds_tarea_featured['validation_match']) == 413

## Model and Tokenizer

El modelo finalmente escogido para hacer el fine-tuning, declaradlo en la variable _model_checkpoint_. Con dicho modelo seleccionado, se pide guardar el modelo y el tokenizador en las variables _model_ y _tokenizer_.

In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_checkpoint = "bert-base-uncased"

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Ejemplo de cómo tokenizar un texto de ejemplo
text = "Example text for tokenization and model testing."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

# Mostrar las logits de la salida del modelo
print(output.logits)

tensor([[ 0.5236, -0.0180,  0.8224]], grad_fn=<AddmmBackward0>)


In [11]:
# Function to tokenize the data
def tokenize_function(examples):
    # Concatenate premise and hypothesis which are typical for NLI tasks
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding="max_length", max_length=128)

# Apply tokenization across the datasets
ds_tarea_featured = {
    "train": ds_tarea['train'].map(tokenize_function, batched=True),
    "validation_match": ds_tarea['validation_match'].map(tokenize_function, batched=True)
}

# Optional: remove columns you don't need for training to optimize memory usage
ds_tarea_featured["train"] = ds_tarea_featured["train"].remove_columns(["premise", "hypothesis", "idx"])
ds_tarea_featured["validation_match"] = ds_tarea_featured["validation_match"].remove_columns(["premise", "hypothesis", "idx"])

Map: 100%|██████████| 413/413 [00:00<00:00, 1720.85 examples/s]


## Fine-tuning

A continuación, de forma libre se pide entrenar un modelo de HuggingFace deseado. Se pide usar un Trainer de HuggingFace que tenga los siguientes argumentos como mínimo (puede haber más argumentos en todas las variables):

In [12]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir='models'
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tarea_featured["train"],
    eval_dataset=ds_tarea_featured["validation_match"],
    tokenizer=tokenizer,
)

A continuación se entrena el modelo

In [13]:
trainer.train()

 10%|▉         | 500/5115 [36:25<5:24:25,  4.22s/it]

{'loss': 0.8422, 'grad_norm': 28.83264923095703, 'learning_rate': 4.511241446725318e-05, 'epoch': 0.29}


 20%|█▉        | 1000/5115 [1:11:49<4:49:29,  4.22s/it]

{'loss': 0.6927, 'grad_norm': 7.9618072509765625, 'learning_rate': 4.0224828934506356e-05, 'epoch': 0.59}


 29%|██▉       | 1500/5115 [1:47:14<4:15:39,  4.24s/it]

...

 98%|█████████▊| 5000/5115 [5:55:42<08:04,  4.21s/it]

{'loss': 0.2346, 'grad_norm': 0.5464977025985718, 'learning_rate': 1.1241446725317694e-06, 'epoch': 2.93}


100%|██████████| 5115/5115 [6:03:51<00:00,  4.27s/it]

{'train_runtime': 21831.2783, 'train_samples_per_second': 1.874, 'train_steps_per_second': 0.234, 'train_loss': 0.45510333966416466, 'epoch': 3.0}


TrainOutput(global_step=5115, training_loss=0.45510333966416466, metrics={'train_runtime': 21831.2783, 'train_samples_per_second': 1.874, 'train_steps_per_second': 0.234, 'total_flos': 2690663588040960.0, 'train_loss': 0.45510333966416466, 'epoch': 3.0})

In [28]:
trainer.save_model("modelo")


## Evaluation

En este apartado, no vamos a entrar esta vez métrics. Lo que se va a pedir, es tomar dos ejemplos del dataset de evaluación. 

Con ambos ejemplos, vamos a ver cómo responden a las preguntas.

In [29]:
sample1 = 55
sample2 = 159

In [30]:
premise1 = ds_tarea['validation_match'][sample1]['premise']
hypothesis1 = ds_tarea['validation_match'][sample1]['hypothesis']
label1 = ds_tarea['validation_match'][sample1]['label']

ds_tarea['validation_match'][sample1]

{'premise': 'Snap Judgment',
 'hypothesis': 'Some judgments about race are made very quickly.',
 'label': 1,
 'idx': 1258}

In [31]:
premise2 = ds_tarea['validation_match'][sample2]['premise']
hypothesis2 = ds_tarea['validation_match'][sample2]['hypothesis']
label2 = ds_tarea['validation_match'][sample2]['label']

ds_tarea['validation_match'][sample2]

{'premise': 'Trying Your Luck',
 'hypothesis': 'Give it a try.',
 'label': 0,
 'idx': 3976}

Aquí se pide hacer la inferencia del modelo entrenado y poner los resultados en las variables _response1_ y _response2_.

In [35]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# Load the model and tokenizer
model_path = "modelo"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [36]:
import torch
# Encode the inputs
inputs1 = tokenizer(premise1, hypothesis1, return_tensors='pt', padding=True, truncation=True)
inputs2 = tokenizer(premise2, hypothesis2, return_tensors='pt', padding=True, truncation=True)

# Make predictions
model.eval()
with torch.no_grad():
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

# Get the predicted labels
predicted_label1 = torch.argmax(outputs1.logits, dim=1).item()
predicted_label2 = torch.argmax(outputs2.logits, dim=1).item()

response1 = {
    "premise": premise1,
    "hypothesis": hypothesis1,
    "true_label": label1,
    "predicted_label": predicted_label1
}

response2 = {
    "premise": premise2,
    "hypothesis": hypothesis2,
    "true_label": label2,
    "predicted_label": predicted_label2
}

print("Response 1:", response1)
print("Response 2:", response2)

Response 1: {'premise': 'Snap Judgment', 'hypothesis': 'Some judgments about race are made very quickly.', 'true_label': 1, 'predicted_label': 1}
Response 2: {'premise': 'Trying Your Luck', 'hypothesis': 'Give it a try.', 'true_label': 0, 'predicted_label': 0}
